## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
# import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#find additional columns to drop based on their frequency
# headers = application_df.columns
# for header in headers:
#   print(application_df[header].value_counts())

10520599     1
626274659    1
630475330    1
630416100    1
630357662    1
            ..
383880377    1
383876652    1
383874900    1
383871370    1
996086871    1
Name: EIN, Length: 34299, dtype: int64
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR  

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME','SPECIAL_CONSIDERATIONS','STATUS'])

In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
bins = application_df['APPLICATION_TYPE'].value_counts()
bins = pd.DataFrame(bins)
bins


,APPLICATION_TYPE
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156
T13,66


In [7]:
# Choose a cutoff value and create a list of application types to be replaced 528
# use the variable name `application_types_to_replace`
application_types_to_replace = []
for index,bin in bins.iterrows():
  if bin[0] < 528:
    application_types_to_replace.append(index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [10]:
#create a valuecounts df
classification_df = pd.DataFrame(classification_counts)

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
for index, row in classification_df.iterrows():
  if row[0] < 1883:
    classifications_to_replace.append(index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.columns

Index(['ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       

In [13]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y =application_df['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
#Count the number of features
features = len(X.columns)


## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 86, activation = 'tanh', input_dim =features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=43, activation ='tanh'))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=43, activation ='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 86)                3526      
                                                                 
 dense_1 (Dense)             (None, 43)                3741      
                                                                 
 dense_2 (Dense)             (None, 43)                1892      
                                                                 
 dense_3 (Dense)             (None, 1)                 44        
                                                                 
Total params: 9203 (35.95 KB)
Trainable params: 9203 (35.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# def create_model(hp):
#   nn = tf.keras.models.Sequential()
#    # Allow kerastuner to decide which activation function to use in hidden layers
#   activation = hp.Choice('activation', ['relu','tanh'])
#    # Allow kerastuner to decide number of neurons in first layer
#   nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
#                          min_value = 1,
#                          max_value = 85,
#                          step = 5), activation = activation, input_dim=features))
#   # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
#   for i in range(hp.Int('num layers', 1, 5)):
#     nn.add(tf.keras.layers.Dense(units=hp.Int('units_'+str(i),
#                           min_value = 1,
#                           max_value= 85,
#                           step = 5), activation = activation))
#   nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
#   #compile the model
#   nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
#   return nn

In [ ]:
#create tuner to search for best hyperparameters
# tuner = kt.Hyperband(create_model, objective="val_accuracy", max_epochs=20, hyperband_iterations=2)

In [ ]:
# run tuner
# tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [17]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [23]:
# Train the model
# from tensorflow.keras.callbacks import EarlyStopping
# stop = EarlyStopping(patience = 3)
nn.fit(X_train_scaled, y_train, epochs=20)


Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5392 - accuracy: 0.7375
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5391 - accuracy: 0.7362
Epoch 3/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5390 - accuracy: 0.7357
Epoch 4/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5381 - accuracy: 0.7372
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5387 - accuracy: 0.7381
Epoch 6/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5381 - accuracy: 0.7376
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5377 - accuracy: 0.7380
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5374 - accuracy: 0.7383
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5373 - accuracy: 0.7381
Epoch 10/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5371 - accuracy: 0.7382

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5526 - accuracy: 0.7299 - 381ms/epoch - 1ms/step
Loss: 0.5526002645492554, Accuracy: 0.729912519454956


In [25]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
